##Libraries

In [ ]:
!pip install torch_geometric

In [ ]:
from scipy.io import loadmat
import numpy as np
from sklearn.metrics import mutual_info_score
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.io import loadmat
from torch_geometric.data import Data

In [ ]:
data1 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 1/Subject12Trial1.mat')['data']
subj1 = data1[:30,:,:]
#subj1 = subj1.reshape(subj1.shape[2],subj1.shape[0],subj1.shape[1])
subj2 = data1[30:,:,:]
#subj2 = subj2.reshape(subj2.shape[2],subj2.shape[0],subj2.shape[1])

data2 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 1/Subject12Trial2.mat')['data']
subj12 = data2[:30,:,:]
#subj12 = subj12.reshape(subj12.shape[2],subj12.shape[0],subj12.shape[1])
subj22 = data2[30:,:,:]
#subj22 = subj22.reshape(subj22.shape[2],subj22.shape[0],subj22.shape[1])

data3 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 1/Subject34Trial1.mat')['data']
subj13 = data3[:30,:,:]
#subj13 = subj13.reshape(subj13.shape[2],subj13.shape[0],subj13.shape[1])
subj23 = data3[30:,:,:]
#subj23 = subj23.reshape(subj23.shape[2],subj23.shape[0],subj23.shape[1])

data4 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 1/Subject34Trial2.mat')['data']
subj14 = data4[:30,:,:]
#subj14 = subj14.reshape(subj14.shape[2],subj14.shape[0],subj14.shape[1])
subj24 = data4[30:,:,:]
#subj24 = subj24.reshape(subj24.shape[2],subj24.shape[0],subj24.shape[1])

data5  = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 2/Subject12Trial1.mat')['data']
subj15 = data5[:30,:,:]
#subj15 = subj15.reshape(subj15.shape[2],subj15.shape[0],subj15.shape[1])
subj25 = data5[30:,:,:]
#subj25 = subj25.reshape(subj25.shape[2],subj25.shape[0],subj25.shape[1])

data6  = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 2/Subject12Trial2.mat')['data']
subj16 = data6[:30,:,:]
#subj16 = subj16.reshape(subj16.shape[2],subj16.shape[0],subj16.shape[1])
subj26 = data6[30:,:,:]
#subj26 = subj26.reshape(subj26.shape[2],subj26.shape[0],subj26.shape[1])

data7  = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 2/Subject34Trial1.mat')['data']
subj17 = data7[:30,:,:]
#subj17 = subj17.reshape(subj17.shape[2],subj17.shape[0],subj17.shape[1])
subj27 = data7[30:,:,:]
#subj27 = subj27.reshape(subj27.shape[2],subj27.shape[0],subj27.shape[1])

data8  = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 2/Subject34Trial2.mat')['data']
subj18 = data8[:30,:,:]
#subj18 = subj18.reshape(subj18.shape[2],subj18.shape[0],subj18.shape[1])
subj28 = data8[30:,:,:]
#subj28 = subj28.reshape(subj28.shape[2],subj28.shape[0],subj28.shape[1])

data = np.concatenate([subj1,subj2,subj12,subj22,subj13,subj23,subj14,subj24,subj15,subj25,
                       subj16,subj26,subj17,subj27,subj18,subj28],axis=2)

data = data.reshape(data.shape[2],data.shape[0],data.shape[1])

# subj1 = data[:30,:,:]
# subj1 = subj1.reshape(subj1.shape[2],subj1.shape[0],subj1.shape[1])
# subj2 = data[30:,:,:]
# subj2 = subj2.reshape(subj2.shape[2],subj2.shape[0],subj2.shape[1])
# subj = data.reshape(data.shape[2],data.shape[0],data.shape[1])
Labels1 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 1/Subject12Trial1Label.mat')['Labels'].T   ##Labels is same for both subjects
Labels2 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 1/Subject12Trial2Label.mat')['Labels'].T
Labels3 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 1/Subject34Trial1Label.mat')['Labels'].T
Labels4 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 1/Subject34Trial2Label.mat')['Labels'].T
Labels5 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 2/Subject12Trial1Label.mat')['Labels'].T
Labels6 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 2/Subject12Trial2Label.mat')['Labels'].T
Labels7 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 2/Subject34Trial1Label.mat')['Labels'].T
Labels8 = loadmat('/content/drive/MyDrive/GRAPH EEG/Experiment 2/Subject34Trial2Label.mat')['Labels'].T

Labels = np.concatenate([Labels1,Labels1,Labels2,Labels2,Labels3,Labels3,Labels4,Labels4,
                         Labels5,Labels5,Labels6,Labels6,Labels7,Labels7,Labels8,Labels8], axis=0)
data.shape, Labels.shape

In [ ]:
np.unique(Labels)

In [ ]:
import numpy as np

def create_trialwise_adjacency_matrices(subj1_data, subj2_data, percentile=55):
    """
    Create an adjacency matrix for each trial between the channels of two subjects
    by applying an adaptive threshold based on the percentile of correlation values.

    Parameters:
    - subj1_data (numpy.ndarray): EEG data for subject 1 of shape (trials, channels, time_points).
    - subj2_data (numpy.ndarray): EEG data for subject 2 of shape (trials, channels, time_points).
    - percentile (float): The percentile to use as a threshold for the adjacency matrix.

    Returns:
    - List of numpy arrays, each of shape (30, 30), representing the adjacency matrix for each trial.
    """
    num_trials = subj1_data.shape[0]  # Number of trials
    num_channels = subj1_data.shape[1]  # Number of channels
    adjacency_matrices = []  # List to store each trial's adjacency matrix

    for trial in range(num_trials):
        # Extract the data for the current trial for each subject
        subj1_trial = subj1_data[trial]  # Shape (30, 1000)
        subj2_trial = subj2_data[trial]  # Shape (30, 1000)

        # Initialize the correlation matrix for this trial
        correlation_matrix = np.zeros((num_channels, num_channels))

        for i in range(num_channels):
            for j in range(num_channels):
                # Calculate the Pearson correlation between channel i of subj1 and channel j of subj2
                correlation = np.corrcoef(subj1_trial[i], subj2_trial[j])[0, 1]
                correlation_matrix[i, j] = correlation

        # Determine the adaptive threshold based on the specified percentile
        threshold = np.percentile(abs(correlation_matrix), percentile)

        # Convert correlation matrix to adjacency matrix based on the threshold
        adjacency_matrix = np.where(abs(correlation_matrix) >= threshold, 1, 0)
        # Compute degree matrix and add it to the adjacency matrix
        # degree_matrix = np.diag(adjacency_matrix.sum(axis=1))
        # adjacency_matrix = degree_matrix - adjacency_matrix

        # Append the calculated adjacency matrix to the list
        adjacency_matrices.append(adjacency_matrix)

    return adjacency_matrices

# Example usage with simulated data
# subj1_data = np.random.rand(29, 30, 1000)  # Simulated data
# subj2_data = np.random.rand(29, 30, 1000)  # Simulated data
adjacency_matrices = create_trialwise_adjacency_matrices(data, data)
print(len(adjacency_matrices))  # Should print the number of trials
print(adjacency_matrices[0].shape)  # Should print (30, 30) for the adjacency matrix shape

In [ ]:
eeg_tensor = np.stack(data)
eeg_tensor = torch.from_numpy(eeg_tensor).clone().detach().requires_grad_(True)
label_tensor = torch.LongTensor(Labels)
#eeg_tensor = torch.from_numpy(s1s2).clone().detach().requires_grad_(True)

In [ ]:
eeg_tensor.size(), label_tensor.size()

In [ ]:
import torch
from torch_geometric.data import Data

def create_pyg_graph_data(eeg_data, adjacency_matrices, labels):
    """
    Convert EEG data and adjacency matrices into PyTorch Geometric graph data objects.

    Parameters:
    - eeg_data (torch.Tensor): The EEG data of shape (trials, channels, time_points).
    - adjacency_matrices (list of np.array): List of adjacency matrices for each trial.
    - labels (torch.Tensor): Labels for each trial, shape (trials, 1).

    Returns:
    - list of Data: A list of PyTorch Geometric Data objects, one per trial.
    """
    graph_data_list = []

    # Correct usage of .size() method
    for i in range(eeg_data.size(0)):
        # Node features: using the EEG data as is, or some transformation can be applied
        x = eeg_data[i]  # Shape: (channels, time_points)

        # Convert the adjacency matrix to edge indices
        adj_matrix = adjacency_matrices[i]
        edge_index = np.vstack(np.nonzero(adj_matrix)).astype(np.int64)

        # Prepare the graph label
        y = labels[i]

        # Create PyTorch Geometric Data object
        data = Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=y)

        graph_data_list.append(data)

    return graph_data_list
graph_data_list = create_pyg_graph_data(eeg_tensor, adjacency_matrices, label_tensor)

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt

# def visualize_graph(data, index):
#     """ Visualize a single graph using NetworkX and matplotlib. """
#     G = nx.Graph()
#     edge_index = data[index].edge_index.numpy()
#     num_nodes = data[index].x.size(0)

#     # Add nodes
#     for node in range(num_nodes):
#         G.add_node(node)

#     # Add edges
#     for i in range(edge_index.shape[1]):
#         source = edge_index[0, i]
#         target = edge_index[1, i]
#         G.add_edge(source, target)

#     # Draw the graph
#     nx.draw(G, with_labels=True, node_color='lightblue')
#     plt.title(f"Graph Visualization for Trial {index}")
#     plt.show()

# # Call this function with any of your graph data objects
# visualize_graph(graph_data_list,index=3)

In [ ]:
# from torch_geometric.data import DataLoader
# import random

# # Assuming graph_data_list is already defined and contains your graph data
# random.shuffle(graph_data_list)  # Shuffle the dataset in-place

# # Set the split ratios for train, validation, and test
# train_ratio = 0.75
# valid_ratio = 0.15
# test_ratio = 0.10  # Ensures that train, valid, and test sum to 1

# # Calculate split indices
# train_index = int(train_ratio * len(graph_data_list))
# valid_index = int((train_ratio + valid_ratio) * len(graph_data_list))

# # Split the dataset
# train_dataset = graph_data_list[:train_index]
# valid_dataset = graph_data_list[train_index:valid_index]
# test_dataset = graph_data_list[valid_index:]

# # Create DataLoaders for each set
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


from torch_geometric.data import DataLoader
import random

# Assuming graph_data_list is already defined
random.shuffle(graph_data_list)  # Shuffle dataset

# Set split ratios
train_ratio = 0.75
valid_ratio = 0.15
test_ratio = 1 - (train_ratio + valid_ratio)  # Ensure sum is exactly 1

# Compute indices properly
num_samples = len(graph_data_list)
train_index = round(train_ratio * num_samples)
valid_index = train_index + round(valid_ratio * num_samples)  # Ensure correct size

# Dataset splits
train_dataset = graph_data_list[:train_index]
valid_dataset = graph_data_list[train_index:valid_index]
test_dataset = graph_data_list[valid_index:]  # Remaining samples go to test

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Debugging Print
print(f"Dataset Sizes → Train: {len(train_dataset)}, Valid: {len(valid_dataset)}, Test: {len(test_dataset)}")

In [ ]:
# import torch
# from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

# class GCN(torch.nn.Module):
#     def __init__(self, num_features, num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(num_features, 512)
#         self.bn1 = BatchNorm(512)  # Batch normalization layer after first GCN layer
#         self.conv2 = GCNConv(512, 256)
#         self.bn2 = BatchNorm(256)  # Batch normalization layer after second GCN layer
#         self.dropout = torch.nn.Dropout(0.5)  # Dropout layer with p=0.5
#         self.conv3 = GCNConv(256, 128)
#         self.bn3 = BatchNorm(128)  # Batch normalization layer after second GCN layer
#         self.out = torch.nn.Linear(128, num_classes)

#     def forward(self, data):
#         x, edge_index, batch = data.x, data.edge_index, data.batch

#         x = self.conv1(x, edge_index)
#         x = self.bn1(x)
#         x = torch.relu(x)
#         x = self.dropout(x)

#         x = self.conv2(x, edge_index)
#         x = self.bn2(x)
#         x = torch.relu(x)
#         x = self.conv3(x, edge_index)
#         x = self.bn3(x)
#         x = torch.relu(x)
#         x = self.dropout(x)

#         x = global_mean_pool(x, batch)  # Global pooling
#         x = self.out(x)

#         return x

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GCN(num_features=1000, num_classes=2).to(device)  # Adjust num_classes based on your labels
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=5e-8)
# criterion = torch.nn.CrossEntropyLoss()

##Main

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

class ImprovedGCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(ImprovedGCN, self).__init__()
        self.conv1 = GCNConv(num_features, 512)
        self.bn1 = BatchNorm(512)
        self.conv2 = GCNConv(512, 256)
        self.bn2 = BatchNorm(256)
        self.conv3 = GCNConv(256, 128)
        self.bn3 = BatchNorm(128)
        self.dropout = torch.nn.Dropout(0.6)
        self.out = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.bn1(x)
        x = self.dropout(x)

        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.bn2(x)

        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = self.bn3(x)
        x = self.dropout(x)

        # 🛠 Fix: Aggregate node embeddings to obtain graph-level features
        x = global_mean_pool(x, batch)  # Converts node features into graph features

       # print(f"Debug: x.shape before Linear Layer: {x.shape}")  # Ensure shape is (batch_size, 128)

        return self.out(x)  # Now passes a proper tensor to Linear layer!

# Training Function

def train(train_loader, model, criterion, optimizer, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(data.y.cpu().numpy())

    return total_loss / len(train_loader), correct / total, precision_score(all_labels, all_preds, average='macro'), recall_score(all_labels, all_preds, average='macro'), f1_score(all_labels, all_preds, average='macro')

# Validation Function

def validate(valid_loader, model, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for data in valid_loader:
            data = data.to(device)
            out = model(data)
            loss = criterion(out, data.y)
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += (pred == data.y).sum().item()
            total += data.y.size(0)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())

    return total_loss / len(valid_loader), correct / total, precision_score(all_labels, all_preds, average='macro'), recall_score(all_labels, all_preds, average='macro'), f1_score(all_labels, all_preds, average='macro')

# Test Function

def test(test_loader, model, device):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == data.y).sum().item()
            total += data.y.size(0)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    return correct / total, precision_score(all_labels, all_preds, average='macro'), recall_score(all_labels, all_preds, average='macro'), f1_score(all_labels, all_preds, average='macro'), cm

# Training Loop with Early Stopping and Learning Rate Scheduler
train_accuracies, val_accuracies, test_accuracies = [], [], []
train_precisions, val_precisions, test_precisions = [], [], []
train_recalls, val_recalls, test_recalls = [], [], []
train_f1s, val_f1s, test_f1s = [], [], []



# Set up model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedGCN(num_features=1000, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

best_val_loss = float('inf')
stopping_patience = 10
counter = 0

for epoch in range(100):
    train_loss, train_acc, train_prec, train_rec, train_f1 = train(train_loader, model, criterion, optimizer, device)
    val_loss, val_acc, val_prec, val_rec, val_f1 = validate(valid_loader, model, criterion, device)
    test_acc, test_prec, test_rec, test_f1, test_cm = test(test_loader, model, device)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    test_accuracies.append(test_acc)

    train_precisions.append(train_prec)
    val_precisions.append(val_prec)
    test_precisions.append(test_prec)

    train_recalls.append(train_rec)
    val_recalls.append(val_rec)
    test_recalls.append(test_rec)

    train_f1s.append(train_f1)
    val_f1s.append(val_f1)
    test_f1s.append(test_f1)

    scheduler.step(val_loss)
    print(f'Epoch {epoch+1}: Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= stopping_patience:
            print("Early stopping triggered.")
            break

# Print Final Metrics
print("\nFinal Performance Statistics:")
def print_metrics(title, metrics):
    print(f'{title}: {np.mean(metrics):.4f} ± {np.std(metrics):.4f}')

print_metrics("Average Train Accuracy", train_accuracies)
print_metrics("Average Validation Accuracy", val_accuracies)
print_metrics("Average Test Accuracy", test_accuracies)
print_metrics("Average Train Precision", train_precisions)
print_metrics("Average Validation Precision", val_precisions)
print_metrics("Average Test Precision", test_precisions)
print_metrics("Average Train Recall", train_recalls)
print_metrics("Average Validation Recall", val_recalls)
print_metrics("Average Test Recall", test_recalls)
print_metrics("Average Train F1 Score", train_f1s)
print_metrics("Average Validation F1 Score", val_f1s)
print_metrics("Average Test F1 Score", test_f1s)


## Total number of parameters

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

## Number of parameters

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

In [ ]:
def model_summary(model):
  print("model_summary")
  print()
  print("Layer_name"+"\t"*7+"Number of Parameters")
  print("="*100)
  model_parameters = [layer for layer in model.parameters() if layer.requires_grad]
  layer_name = [child for child in model.children()]
  j = 0
  total_params = 0
  print("\t"*10)
  for i in layer_name:
    print()
    param = 0
    try:
      bias = (i.bias is not None)
    except:
      bias = False
    if not bias:
      param =model_parameters[j].numel()+model_parameters[j+1].numel()
      j = j+2
    else:
      param =model_parameters[j].numel()
      j = j+1
    print(str(i)+"\t"*3+str(param))
    total_params+=param
  print("="*100)
  print(f"Total Params:{total_params}")

model_summary(model)